<h1 align=center><font size = 5>Identifying Suitable Areas for a Cycle Shop in London</font></h1>
<h2 align=center><font size = 5>Part 1 - Acquisition and Cleaning of Data</font></h2>

## Introduction

In this notebook we will be acquiring and preparing our data ready for EDA.  Our data sources are

 - List of London Boroughs including area and population size from *Wikipedia*
 - JSON file with the boundaries of each of Londons boroughs from *http://skgrange.github.io/data.html*
 - JSON file with the current cycle routes from *Transport for Londons Open Data Portal* (not yet available in API)
 - List of cycle shops in London using the *FourSquare API*



## Notebook Setup

In [1]:
#!pip install Beautifulsoup4 
#!pip install lxml

import pandas as pd
import numpy as np
#import folium
import json
import urllib.request
from bs4 import BeautifulSoup
import lxml
import requests
from pandas.core.common import flatten

In [2]:
# This function is called to download any content from the web (Mac), giving a status update throughout
# import urllib.request

def Download_Progress(block_num, block_size, total_size):
    downloaded = block_num * block_size
    progress = int((downloaded/total_size)*100)
    if progress > 100:
        progress = 100
    print ('\r',"Download Progress",str(progress),"%", end = '')

In [3]:
# This function splits a list with n elements per list.  It's used after we flatten the route co-ordinates due to multiple levels of list and need to rebuild them as a single list.
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [4]:
#This function is used to determine if a point is within a json boundary.  It will take the boundary details from the geoData.json file and compare it against the coordinates supplied.  Boolean answer with True as falling within the boundary and false outside.

def is_point_in_path(x: int, y: int, poly) -> bool:
    """Determine if the point is in the path.

    Args:
      x -- The x coordinates of point.
      y -- The y coordinates of point.
      poly -- a list of tuples [(x, y), (x, y), ...]

    Returns:
      True if the point is in the path.
    """
    num = len(poly)
    i = 0
    j = num - 1
    c = False
    for i in range(num):
        if ((poly[i][1] > y) != (poly[j][1] > y)) and \
                (x < poly[i][0] + (poly[j][0] - poly[i][0]) * (y - poly[i][1]) /
                                  (poly[j][1] - poly[i][1])):
            c = not c
        j = i
    return c

## London Boroughs, Primary Dataset

In [5]:
#!pip install Beautifulsoup4 
#!pip install lxml

#from bs4 import BeautifulSoup
#import lxml
#import requests

url = 'https://en.wikipedia.org/wiki/List_of_London_boroughs'
source = requests.get(url).text

soup = BeautifulSoup(source, 'lxml')

codeTable = soup.find('table')

entries = [] # Used for splitting each row

# split the text
print("Splitting each table entry")
for td in codeTable.find_all('tr'):
    entries.append((td.text).split('\n'))

print("First 3 Entries")
display(entries[0:3])


Splitting each table entry
First 3 Entries


[['',
  'Borough',
  '',
  'Inner',
  '',
  'Status',
  '',
  'Local authority',
  '',
  'Political control',
  '',
  'Headquarters',
  '',
  'Area (sq mi)',
  '',
  'Population (2013 est)[1]',
  '',
  'Co-ordinates',
  '',
  ' Nr. in map ',
  ''],
 ['',
  'Barking and Dagenham [note 1]',
  '',
  '',
  '',
  '',
  '',
  'Barking and Dagenham London Borough Council',
  '',
  'Labour',
  '',
  'Town Hall, 1 Town Square',
  '',
  '13.93',
  '',
  '194,352',
  '',
  '51°33′39″N 0°09′21″E\ufeff / \ufeff51.5607°N 0.1557°E\ufeff / 51.5607; 0.1557\ufeff (Barking and Dagenham)',
  '',
  '25',
  ''],
 ['',
  'Barnet',
  '',
  '',
  '',
  '',
  '',
  'Barnet London Borough Council',
  '',
  'Conservative',
  '',
  'Barnet House, 2 Bristol Avenue, Colindale',
  '',
  '33.49',
  '',
  '369,088',
  '',
  '51°37′31″N 0°09′06″W\ufeff / \ufeff51.6252°N 0.1517°W\ufeff / 51.6252; -0.1517\ufeff (Barnet)',
  '',
  '31',
  '']]

In [6]:
# The coordinates have a BOM (\ufeff) character that needs to be cleaned out 

cleanCoordinates = []

for entry in entries:
    cleanCoordinates.append(str.replace(entry[17],u'\ufeff', ''))

cleanCoordinates.pop(0) #Drop column name
cleanCoordinates

['51°33′39″N 0°09′21″E / 51.5607°N 0.1557°E / 51.5607; 0.1557 (Barking and Dagenham)',
 '51°37′31″N 0°09′06″W / 51.6252°N 0.1517°W / 51.6252; -0.1517 (Barnet)',
 '51°27′18″N 0°09′02″E / 51.4549°N 0.1505°E / 51.4549; 0.1505 (Bexley)',
 '51°33′32″N 0°16′54″W / 51.5588°N 0.2817°W / 51.5588; -0.2817 (Brent)',
 '51°24′14″N 0°01′11″E / 51.4039°N 0.0198°E / 51.4039; 0.0198 (Bromley)',
 '51°31′44″N 0°07′32″W / 51.5290°N 0.1255°W / 51.5290; -0.1255 (Camden)',
 '51°22′17″N 0°05′52″W / 51.3714°N 0.0977°W / 51.3714; -0.0977 (Croydon)',
 '51°30′47″N 0°18′32″W / 51.5130°N 0.3089°W / 51.5130; -0.3089 (Ealing)',
 '51°39′14″N 0°04′48″W / 51.6538°N 0.0799°W / 51.6538; -0.0799 (Enfield)',
 '51°29′21″N 0°03′53″E / 51.4892°N 0.0648°E / 51.4892; 0.0648 (Greenwich)',
 '51°32′42″N 0°03′19″W / 51.5450°N 0.0553°W / 51.5450; -0.0553 (Hackney)',
 '51°29′34″N 0°14′02″W / 51.4927°N 0.2339°W / 51.4927; -0.2339 (Hammersmith and Fulham)',
 '51°36′00″N 0°06′43″W / 51.6000°N 0.1119°W / 51.6000; -0.1119 (Haringey)',
 '51

In [7]:
# Conert these entries to a Dataframe

borough = []
area = []
population = []

for entry in entries:
    borough.append(entry[1])
    area.append(entry[13])
    population.append(entry[15])


# Dicionary for DF Column Names
dfDict = {'Borough' : borough, 'Area' : area, 'Population' : population}


dfLondon = pd.DataFrame(dfDict) # Empty DF with Columns
dfLondon.drop([0], inplace = True) # First row had column names, drop these

# Add in our clean co-ordinates
dfLondon['Coordinates'] = cleanCoordinates
dfLondon

print()
print("Converted to DataFrame")
display(dfLondon)


Converted to DataFrame


,Borough,Area,Population,Coordinates
1,Barking and Dagenham [note 1],13.93,"194,352",51°33′39″N 0°09′21″E / 51.5607°N 0.1557°E / 51...
2,Barnet,33.49,"369,088",51°37′31″N 0°09′06″W / 51.6252°N 0.1517°W / 51...
3,Bexley,23.38,"236,687",51°27′18″N 0°09′02″E / 51.4549°N 0.1505°E / 51...
4,Brent,16.70,"317,264",51°33′32″N 0°16′54″W / 51.5588°N 0.2817°W / 51...
5,Bromley,57.97,"317,899",51°24′14″N 0°01′11″E / 51.4039°N 0.0198°E / 51...
6,Camden,8.40,"229,719",51°31′44″N 0°07′32″W / 51.5290°N 0.1255°W / 51...
7,Croydon,33.41,"372,752",51°22′17″N 0°05′52″W / 51.3714°N 0.0977°W / 51...
8,Ealing,21.44,"342,494",51°30′47″N 0°18′32″W / 51.5130°N 0.3089°W / 51...
9,Enfield,31.74,"320,524",51°39′14″N 0°04′48″W / 51.6538°N 0.0799°W / 51...
10,Greenwich [note 2],18.28,"264,008",51°29′21″N 0°03′53″E / 51.4892°N 0.0648°E / 51...


## Clean Up Borough Data


In [8]:
# Remove note from Borough name
dfLondon.Borough.replace(regex=r' \[note \d]', value = "", inplace=True) 

# Only keep the central set of co-ordinates
dfLondon.Coordinates.replace(regex=r'^(.*?)\/(.*?)\/',value="", inplace=True)
dfLondon.Coordinates.replace(regex=r'\((.*?)\)$',value=r'', inplace=True)

# Split the Coordinates
dfLondon['Latitude'], dfLondon['Longitude'] = dfLondon['Coordinates'].str.split(';', 1).str
dfLondon

,Borough,Area,Population,Coordinates,Latitude,Longitude
1,Barking and Dagenham,13.93,"194,352",51.5607; 0.1557,51.5607,0.1557
2,Barnet,33.49,"369,088",51.6252; -0.1517,51.6252,-0.1517
3,Bexley,23.38,"236,687",51.4549; 0.1505,51.4549,0.1505
4,Brent,16.70,"317,264",51.5588; -0.2817,51.5588,-0.2817
5,Bromley,57.97,"317,899",51.4039; 0.0198,51.4039,0.0198
6,Camden,8.40,"229,719",51.5290; -0.1255,51.5290,-0.1255
7,Croydon,33.41,"372,752",51.3714; -0.0977,51.3714,-0.0977
8,Ealing,21.44,"342,494",51.5130; -0.3089,51.5130,-0.3089
9,Enfield,31.74,"320,524",51.6538; -0.0799,51.6538,-0.0799
10,Greenwich,18.28,"264,008",51.4892; 0.0648,51.4892,0.0648


In [9]:
import numpy as np
print("\n Original Data Types")
display(dfLondon.dtypes)

dfLondon.Area = dfLondon.Area.astype(float)

#Remove commas from population otherwise we struggle to convert to float
dfLondon.Population.replace(regex=r'\,', value = "", inplace=True)
dfLondon.Population = dfLondon.Population.astype(int)
dfLondon.Latitude = dfLondon.Latitude.astype(float)
dfLondon.Longitude = dfLondon.Longitude.astype(float)

print("\n Converted Data Types")
display(dfLondon.dtypes)

print("\nLet's also calculate a radius so we can get a more accurate picture of locality")
print("\nWe will use the area to calculate a circular radius from the boroughs central co-ordinates")
dfLondon['Radius'] = (np.sqrt((dfLondon.Area/np.pi))*1540)

# We need to reset the index as the first entry was column names
dfLondon.reset_index(drop=True, inplace=True)

display(dfLondon)


 Original Data Types


Borough        object
Area           object
Population     object
Coordinates    object
Latitude       object
Longitude      object
dtype: object


 Converted Data Types


Borough         object
Area           float64
Population       int64
Coordinates     object
Latitude       float64
Longitude      float64
dtype: object


Now we can add a density column

Let's also calculate a radius so we can get a more accurate picture of locality

We will use the area to calculate a circular radius from the boroughs central co-ordinates


,Borough,Area,Population,Coordinates,Latitude,Longitude,Density,Radius
0,Barking and Dagenham,13.93,194352,51.5607; 0.1557,51.5607,0.1557,13952.05,3242.808799
1,Barnet,33.49,369088,51.6252; -0.1517,51.6252,-0.1517,11020.84,5028.093653
2,Bexley,23.38,236687,51.4549; 0.1505,51.4549,0.1505,10123.48,4201.148547
3,Brent,16.70,317264,51.5588; -0.2817,51.5588,-0.2817,18997.84,3550.618569
4,Bromley,57.97,317899,51.4039; 0.0198,51.4039,0.0198,5483.85,6615.267871
5,Camden,8.40,229719,51.5290; -0.1255,51.5290,-0.1255,27347.50,2518.172214
6,Croydon,33.41,372752,51.3714; -0.0977,51.3714,-0.0977,11156.90,5022.084576
7,Ealing,21.44,342494,51.5130; -0.3089,51.5130,-0.3089,15974.53,4023.075426
8,Enfield,31.74,320524,51.6538; -0.0799,51.6538,-0.0799,10098.42,4894.961110
9,Greenwich,18.28,264008,51.4892; 0.0648,51.4892,0.0648,14442.45,3714.786685


In [10]:
# Backup latest updates
dfLondon.to_csv (r'dfLondon.csv',index = False, header = True)

dfLondon = pd.read_csv("dfLondon.csv")
dfLondon

,Borough,Area,Population,Coordinates,Latitude,Longitude,Density,Radius
0,Barking and Dagenham,13.93,194352,51.5607; 0.1557,51.5607,0.1557,13952.05,3242.808799
1,Barnet,33.49,369088,51.6252; -0.1517,51.6252,-0.1517,11020.84,5028.093653
2,Bexley,23.38,236687,51.4549; 0.1505,51.4549,0.1505,10123.48,4201.148547
3,Brent,16.70,317264,51.5588; -0.2817,51.5588,-0.2817,18997.84,3550.618569
4,Bromley,57.97,317899,51.4039; 0.0198,51.4039,0.0198,5483.85,6615.267871
5,Camden,8.40,229719,51.5290; -0.1255,51.5290,-0.1255,27347.50,2518.172214
6,Croydon,33.41,372752,51.3714; -0.0977,51.3714,-0.0977,11156.90,5022.084576
7,Ealing,21.44,342494,51.5130; -0.3089,51.5130,-0.3089,15974.53,4023.075426
8,Enfield,31.74,320524,51.6538; -0.0799,51.6538,-0.0799,10098.42,4894.961110
9,Greenwich,18.28,264008,51.4892; 0.0648,51.4892,0.0648,14442.45,3714.786685


## Download the Borough Boundaries


In [11]:
url = "https://skgrange.github.io/www/data/london_boroughs.json"
lclfile = "geoData.json"
urllib.request.urlretrieve(url, lclfile, reporthook=Download_Progress)
print (" \nFinished")

 Download Progress 100 % 
Finished


In [12]:
dfLondon

,Borough,Area,Population,Coordinates,Latitude,Longitude,Density,Radius
0,Barking and Dagenham,13.93,194352,51.5607; 0.1557,51.5607,0.1557,13952.05,3242.808799
1,Barnet,33.49,369088,51.6252; -0.1517,51.6252,-0.1517,11020.84,5028.093653
2,Bexley,23.38,236687,51.4549; 0.1505,51.4549,0.1505,10123.48,4201.148547
3,Brent,16.70,317264,51.5588; -0.2817,51.5588,-0.2817,18997.84,3550.618569
4,Bromley,57.97,317899,51.4039; 0.0198,51.4039,0.0198,5483.85,6615.267871
5,Camden,8.40,229719,51.5290; -0.1255,51.5290,-0.1255,27347.50,2518.172214
6,Croydon,33.41,372752,51.3714; -0.0977,51.3714,-0.0977,11156.90,5022.084576
7,Ealing,21.44,342494,51.5130; -0.3089,51.5130,-0.3089,15974.53,4023.075426
8,Enfield,31.74,320524,51.6538; -0.0799,51.6538,-0.0799,10098.42,4894.961110
9,Greenwich,18.28,264008,51.4892; 0.0648,51.4892,0.0648,14442.45,3714.786685


In [13]:
# Add the feature id for the boundaries of each borough into our dfLondon dataframe.  We can use this later to determine if a supplied coordinate is within a boundary

map_geo = "geoData.json"

with open(map_geo) as f:
    loadPoly = json.load(f)

for i in range(0,33):
  boroughName = loadPoly['features'][i]['properties']['name']
  dfLondon.loc[dfLondon.Borough == boroughName,'GeoFeature']=i

# Set as Integer value
dfLondon.GeoFeature = dfLondon.GeoFeature.astype(int)

# Display table in order to check all values are complete and there are no duplicates
dfLondon.sort_values(by=['GeoFeature'])

,Borough,Area,Population,Coordinates,Latitude,Longitude,Density,Radius,GeoFeature
19,Kingston upon Thames,14.38,166793,51.4085; -0.3064,51.4085,-0.3064,11598.96,3294.770945,0
6,Croydon,33.41,372752,51.3714; -0.0977,51.3714,-0.0977,11156.90,5022.084576,1
4,Bromley,57.97,317899,51.4039; 0.0198,51.4039,0.0198,5483.85,6615.267871,2
16,Hounslow,21.61,262407,51.4746; -0.3680,51.4746,-0.3680,12142.85,4038.993627,3
7,Ealing,21.44,342494,51.5130; -0.3089,51.5130,-0.3089,15974.53,4023.075426,4
14,Havering,43.35,242080,51.5812; 0.1837,51.5812,0.1837,5584.31,5720.583583,5
15,Hillingdon,44.67,286806,51.5441; -0.4760,51.5441,-0.4760,6420.55,5807.025869,6
13,Harrow,19.49,243372,51.5898; -0.3346,51.5898,-0.3346,12487.02,3835.762456,7
3,Brent,16.70,317264,51.5588; -0.2817,51.5588,-0.2817,18997.84,3550.618569,8
1,Barnet,33.49,369088,51.6252; -0.1517,51.6252,-0.1517,11020.84,5028.093653,9


## Download Cycle Routes

In [14]:
url = "https://cycling.data.tfl.gov.uk/CycleRoutes/CycleRoutes.json"
lclfile = "cycleRoutes.json"
urllib.request.urlretrieve(url, lclfile, reporthook=Download_Progress)
print (" \nFinished")

 Download Progress 100 % 
Finished


In [15]:
#Build three objects that include the co-ordinates for all cycle routes.  These have been split between open cycle routes, those that are currently in progress and any routes planned for the future.  We can use these to see how much of each type of route passes within each boroughs boundaries

cycle_geo = "cycleRoutes.json"

with open(cycle_geo) as f:
    cycleRoutes = json.load(f)

openRoute = []
inProgressRoute = []
plannedRoute = []

for i in range(0,len(cycleRoutes['features'])):
    if cycleRoutes['features'][i]['properties']['Status'] == 'Open':
        addRoute = cycleRoutes['features'][i]['geometry']['coordinates']
        
        t = list(flatten(addRoute)) #We have multiple levels of lists.  Will flatten and rebuild as a single one
        t2 = list(chunks(t,3))
        
        for x in range (0,len(t2)):
            openRoute.append(t2[x])


    if cycleRoutes['features'][i]['properties']['Status'] == 'In Progress':
        addRoute = cycleRoutes['features'][i]['geometry']['coordinates']

        t = list(flatten(addRoute)) #We have multiple levels of lists.  Will flatten and rebuild as a single one
        t2 = list(chunks(t,3))
        
        for x in range (0,len(t2)):
            inProgressRoute.append(t2[x])

    if cycleRoutes['features'][i]['properties']['Status'] == 'Planned':
        addRoute = cycleRoutes['features'][i]['geometry']['coordinates']

        t = list(flatten(addRoute)) #We have multiple levels of lists.  Will flatten and rebuild as a single one
        t2 = list(chunks(t,3))
        
        for x in range (0,len(t2)):
            plannedRoute.append(t2[x])

print("Length of Open Routes is {}".format(len(openRoute)))
print("Length of In Progress Routes is {}".format(len(inProgressRoute)))
print("Length of Planned Routes is {}".format(len(plannedRoute)))
print(len(openRoute)+len(inProgressRoute)+len(plannedRoute))


Length of Open Routes is 22658
Length of In Progress Routes is 6795
Length of Planned Routes is 8891
38344


In [16]:
routeCounts = []

routeLists = [openRoute, inProgressRoute, plannedRoute]


import json

with open(map_geo) as f:
    loadBorough = json.load(f)

dfRouteDensity = pd.DataFrame(columns = ['GeoFeature','openRoute', 'inProgressRoute', 'plannedRoute'])


for i in range (0,32): # Cycle through each borough
    dfRouteDensity.at[i,'GeoFeature'] = i
    temp = loadBorough['features'][i]['geometry']['coordinates']
    cList = ''
    for t in range(0,len(temp)): # Cycle the type of route
        boroughPoly = tuple(temp[t][0])
        for routeList in routeLists:
            if cList == 'openRoute':
                cList = 'inProgressRoute'
            elif cList == 'inProgressRoute':
                cList = 'plannedRoute'
            else:
                cList = 'openRoute'
            BDCount = 0
            for n in range(len(routeList)): # For each set of co-ordinates check if it is within the boundary of the borough
                x = (routeList[n][0])
                y = (routeList[n][1])
                if is_point_in_path(x,y,boroughPoly) == True:
                    BDCount += 1
            #print(i,cList, BDCount)
            dfRouteDensity.at[i,cList] = BDCount # Add count value to our dfRouteDensity Dataframa

dfRouteDensity


,GeoFeature,openRoute,inProgressRoute,plannedRoute
0,0,417,891,187
1,1,89,0,174
2,2,0,286,413
3,3,0,0,881
4,4,628,565,36
5,5,0,0,0
6,6,129,36,167
7,7,0,0,0
8,8,426,5,0
9,9,0,0,0


## Use FourSquare API to download the list of Cycle Shops by each Borough in London

In [17]:
sys.path.append('../') # Credentials file is in a sub directory so allow it to be loaded 
import credentials
# Below 2 lines required if credentials had been updated
#from importlib import reload
#reload(credentials)

CLIENT_ID = credentials.foursquare['client_id']# your Foursquare ID
CLIENT_SECRET = credentials.foursquare['client_secret'] # your Foursquare Secret
ACCESS_TOKEN = credentials.foursquare['access_token']
VERSION = '20180605'

print("FourSquare Credentials Retrieved")

FourSquare Credentials Retrieved


In [18]:

# Function to get a list of cycle shops by the category ID "4bf58dd8d48988d115951735" taken from 
#https://api.foursquare.com/v2/venues/categories 

def getNearbyVenues(names, latitudes, longitudes, radius):

    categoryId = "4bf58dd8d48988d115951735"
    limit = 50 #API explorer shows this is max for each seach
    
    cycleShop_list=[]
    for name, lat, lng, radius in zip(names, latitudes, longitudes, radius):
        print('\r',name, end='')
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&time=any&day=any&sortByDistance=1&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            categoryId,
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        #return only relevant information for each nearby venue
        cycleShop_list.append([(
            name,  
            lat, 
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng']) for v in results])
    #print(cycleShop_list)

    dfCycleShops = pd.DataFrame([item for cycleShop in cycleShop_list for item in cycleShop])
    print(dfCycleShops)
    dfCycleShops.columns = ['Borough', 
                  'PostalCode Latitude',
                  'PostalCode Longitude', 
                  'Shop Name', 
                  'Shop Latitude', 
                  'Shop Longitude']
    
    return(dfCycleShops)

In [19]:
# Let's get our list of cycle shops 

dfCycleShops = getNearbyVenues(names=dfLondon['Borough'],
                                   latitudes=dfLondon['Latitude'],
                                   longitudes=dfLondon['Longitude'],
                                   radius=dfLondon['Radius']
                                  )

print(" \nThe shape of the Cycling Shops dataframe is {}".format(dfCycleShops.shape))

print("\nThe following table shows how many shops were returned for each Borough \n")
dfDistribution = pd.DataFrame(dfCycleShops[['Borough','Shop Name']].groupby('Borough').count())
dfDistribution

Westminster                        0        1       2                            3  \
0    Barking and Dagenham  51.5607  0.1557              The Cycle Store   
1    Barking and Dagenham  51.5607  0.1557            Cycles Uk Romford   
2                  Barnet  51.6252 -0.1517  Bike and Run (Road, TT etc)   
3                  Barnet  51.6252 -0.1517           NRV Residents' Gym   
4                  Barnet  51.6252 -0.1517        future cycle training   
..                    ...      ...     ...                          ...   
208           Westminster  51.4973 -0.1372                Balfe's Bikes   
209           Westminster  51.4973 -0.1372            Brompton Junction   
210           Westminster  51.4973 -0.1372                 Evans Cycles   
211           Westminster  51.4973 -0.1372                   Velorution   
212           Westminster  51.4973 -0.1372     TfL Santander Cycle Hire   

             4         5  
0    51.563217  0.117297  
1    51.579158  0.181277  
2    51

,Shop Name
Borough,
Barking and Dagenham,2
Barnet,9
Bexley,5
Brent,3
Bromley,2
Camden,26
Croydon,4
Ealing,6
Enfield,2


## Merge all data sources into the one table and tidy up data

In [24]:
#We don't need this column anymore
dfLondon.drop(columns = "Coordinates", inplace = True)

# Add in the Cycle routes Data
dfLondon = dfLondon.merge(dfRouteDensity, on="GeoFeature")

# Convert the columns to numeric
cols = ['openRoute', 'inProgressRoute', 'plannedRoute']
dfLondon[cols] = dfLondon[cols].apply(pd.to_numeric, errors='coerce', axis=1)

# Add a total routes column
dfLondon['Total Routes'] = round(dfLondon.openRoute + dfLondon.inProgressRoute + dfLondon.plannedRoute,2)

#Add in the count of Cycle Shops for each borough
dfLondon = pd.merge(dfLondon, dfDistribution, on='Borough')

# Rename Columns ot more meaningful names
dfLondon.rename(columns={"Shop Name" : "Shop Count",'openRoute': 'Open Routes', 
'inProgressRoute' : 'In Progress Routes', 'plannedRoute' : "Planned Routes"}, inplace=True)

#We don't need these column anymore
dfLondon.drop(columns = ['GeoFeature'], inplace = True)

# Add a routes/shops column
dfLondon['Route/Shop Ratio'] = round(dfLondon['Total Routes']/dfLondon['Shop Count'],2)

In [25]:


# Backup latest updates
dfLondon.to_csv (r'dfLondon.csv',index = False, header = True)
dfCycleShops.to_csv (r'dfCycleShops.csv',index = False, header = True)


### Three Python Notebooks

1. Data Collection and Cleaning - https://github.com/moaeedsajid/Coursera_Capstone/blob/master/Capstone_1.ipynb
2. Exploratory Data Analysis - https://github.com/moaeedsajid/Coursera_Capstone/blob/master/Capstone_2.ipynb
3. Modelling and Results - https://github.com/moaeedsajid/Coursera_Capstone/blob/master/Capstone_3.ipynb